In [1]:
from urllib.request import urlopen
import json
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf

In [2]:
def get_fmp_jsondata(url):
    response = urlopen(url)
    data = response.read().decode("utf-8")
    return json.loads(data)

In [3]:
def get_financial_records(ticker):
    def get_fmp_jsondata(url):
        response = urlopen(url)
        data = response.read().decode('utf-8')
        return json.loads(data)

    income_statement_url = f'https://financialmodelingprep.com/api/v3/financials/income-statement/{ticker}'
    balance_sheet_url = f'https://financialmodelingprep.com/api/v3/financials/balance-sheet-statement/{ticker}'
    cash_flow_url = f'https://financialmodelingprep.com/api/v3/financials/cash-flow-statement/{ticker}'
    enterprise_values_url = f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}'
    price_url = f'https://financialmodelingprep.com/api/v3/stock/real-time-price/{ticker}'
    
    #query APIs for statement values
    try:
        income_statement = get_fmp_jsondata(income_statement_url)
        balance_sheet = get_fmp_jsondata(balance_sheet_url)
        statement_of_cash_flows = get_fmp_jsondata(cash_flow_url)
        enterprise_values = get_fmp_jsondata(enterprise_values_url)
        price = get_fmp_jsondata(price_url)
    except:
        print(f"Could not query for {ticker} on Financial Modeling Prep's API.")

    try:
        price = price['price']
    except:
        print(f'Could not process price for {ticker}.')
        price = float('NaN')
    
    #coerce to pandas dataframes 
    try:
        income_statement = pd.DataFrame(income_statement['financials'])
        balance_sheet = pd.DataFrame(balance_sheet['financials'])
        statement_of_cash_flows = pd.DataFrame(statement_of_cash_flows['financials'])
        enterprise_values = pd.DataFrame(enterprise_values['enterpriseValues'])
    except:
       print(f'Unable to process {ticker} as pandas dataframe.')
    
    #change statement dataframes to datetime and float values
    def coerce_statement_df(df):
        df['date'] = pd.to_datetime(df['date'])
        df.loc[:, df.columns != 'date'] = df.loc[:, df.columns != 'date'].astype(float)
        return df

    try:
        income_statement = coerce_statement_df(income_statement)
        balance_sheet = coerce_statement_df(balance_sheet)
        statement_of_cash_flows = coerce_statement_df(statement_of_cash_flows)
        enterprise_values = coerce_statement_df(enterprise_values)
    except: 
        print(f'Could not coerce dataframes to specified values for {ticker}')

    return income_statement, balance_sheet, statement_of_cash_flows, enterprise_values, price

In [4]:
ticker = 'aapl'

In [5]:
income_statement, balance_sheet, statement_of_cash_flows, enterprise_values, price = get_financial_records(ticker)

In [6]:
def target_share_prices(income_statement, balance_sheet, statement_of_cash_flows, enterprise_values, price, ticker):
    print(ticker)    
    #Get Shares Outstanding
    try:
        shares_outstanding = enterprise_values['Number of Shares'][0]
    except ValueError:
        print(f'---Could not get shares outstanding query for {ticker} from Financial Modeling Prep. Trying Yahoo Finance...')
        shares_outstanding = yf.Ticker(ticker).info['sharesOutstanding']
    except:
        print('---Could not get number of outstanding shares from Yahoo Finance...')
        shares_outstanding = float('NaN')
    
    #calculate ten cap price
    def tc_price(income_statement, balance_sheet, statement_of_cash_flows, shares_outstanding):
        try:
            net_income = income_statement['Net Income'][0]
            net_receivables = balance_sheet['Receivables'][0]-balance_sheet['Receivables'][1]
            net_payables = balance_sheet['Payables'][0]-balance_sheet['Payables'][1]
            income_tax = income_statement['Income Tax Expense'][0]
            capex = statement_of_cash_flows['Capital Expenditure'][0]
        except:
            print('---Unable to calculate Ten Cap price.')
        try:
            ten_cap_price = net_income + net_receivables + net_payables + income_tax + capex
            ten_cap_price = round((ten_cap_price*10)/shares_outstanding,2)
        except:
            ten_cap_price = float('NaN')
        return ten_cap_price

    #calculate payback time price
    def pbt_price(statement_of_cash_flows, shares_outstanding):
        try:
            pbt_price = round((statement_of_cash_flows['Free Cash Flow'][0]*(1.16*8))/(enterprise_values['Number of Shares'][0]),2)
        except:
            print('---Unable to calculate Payback Time price.')
            pbt_price = float('NaN')
        return pbt_price

    ten_cap_price = tc_price(income_statement, balance_sheet, statement_of_cash_flows, shares_outstanding)
    payback_time_price = pbt_price(statement_of_cash_flows, shares_outstanding)

    return price, ten_cap_price, payback_time_price

In [7]:
target_share_prices(income_statement, balance_sheet, statement_of_cash_flows, enterprise_values, price, ticker)

aapl


(275.16, 85.56, 110.3)

In [55]:
def metric_growth(metric, df):
    total_years = df.count(axis='rows')[metric]
    total_years -= 1
    new_column_index = df.columns.get_loc(metric) + 1
    
    metric_growth = []
    for year in range(total_years):
        last_year = year + 1
        metric_growth_rate = round(((df[metric][year] - df[metric][last_year]) / df[metric][last_year]), 3)
        metric_growth.append(metric_growth_rate)
    metric_growth.append(float('NaN'))
    index = df.columns.get_loc(metric) + 1
    try:
        df.insert(index, f'{metric} Growth Rate', metric_growth, True)
    except ValueError:
        df.insert(index, f'{metric} Growth Rate', pd.Series(metric_growth), True)
    return df

In [34]:
#calculate and insert book value per share (BVPS)
metric = 'Total shareholders equity'
total_years = balance_sheet.count(axis='rows')[metric]
index = balance_sheet.columns.get_loc(metric) + 1
bvps = []
for year in range(total_years):
    try:
        bvps.append((balance_sheet[metric][year] / enterprise_values['Number of Shares'][year]))
    except KeyError: 
        bvps.append(float('NaN'))
balance_sheet.insert(index, 'Book Value per Share', bvps, False)

In [56]:
metric_growth('EPS', income_statement)
metric_growth('Revenue', income_statement)
metric_growth('Operating Income', income_statement)
metric_growth('Cash and cash equivalents', balance_sheet)
metric_growth('Book Value per Share', balance_sheet)
metric_growth('Free Cash Flow', statement_of_cash_flows)

,date,Cash and cash equivalents,Cash and cash equivalents Growth Rate,Short-term investments,Cash and short-term investments,Receivables,Inventories,Total current assets,"Property, Plant & Equipment Net",Goodwill and Intangible Assets,...,Total liabilities,Other comprehensive income,Retained earnings (deficit),Total shareholders equity,Book Value per Share,Book Value per Share Growth Rate,Investments,Net Debt,Other Assets,Other Liabilities
0,2019-09-28,4.884400e+10,0.885,5.171300e+10,1.005570e+11,4.580400e+10,4.106000e+09,1.628190e+11,3.737800e+10,0.000000e+00,...,2.480280e+11,-5.840000e+08,4.589800e+10,9.048800e+10,18.260568,-0.111,1.570540e+11,7.490000e+09,1.235200e+10,4.324200e+10
1,2018-09-29,2.591300e+10,0.277,4.038800e+10,6.630100e+10,4.899500e+10,3.956000e+09,1.313390e+11,4.130400e+10,0.000000e+00,...,2.585780e+11,-3.454000e+09,7.040000e+10,1.071470e+11,20.537096,-0.162,2.111870e+11,4.818200e+10,1.208700e+10,3.929300e+10
2,2017-09-30,2.028900e+10,-0.010,5.389200e+10,7.418100e+10,3.567300e+10,4.855000e+09,1.286450e+11,3.378300e+10,0.000000e+00,...,2.412720e+11,-1.500000e+08,9.833000e+10,1.340470e+11,24.502177,0.099,2.486060e+11,4.149900e+10,1.393600e+10,3.809900e+10
3,2016-09-24,2.048400e+10,-0.030,4.667100e+10,6.715500e+10,2.929900e+10,2.132000e+09,1.068690e+11,2.701000e+10,8.620000e+09,...,1.934370e+11,6.340000e+08,9.636400e+10,1.282490e+11,22.290912,0.137,2.171010e+11,1.987700e+10,8.283000e+09,3.010700e+10
4,2015-09-26,2.112000e+10,0.526,2.048100e+10,4.160100e+10,3.034300e+10,2.349000e+09,8.937800e+10,2.247100e+10,9.009000e+09,...,1.709900e+11,-3.450000e+08,9.228400e+10,1.193550e+11,19.612782,NaN,1.845460e+11,2.272700e+10,1.508500e+10,3.412100e+10
5,2014-09-27,1.384400e+10,-0.029,1.123300e+10,2.507700e+10,2.721900e+10,2.111000e+09,6.853100e+10,2.062400e+10,8.758000e+09,...,1.202920e+11,1.082000e+09,8.715200e+10,1.115470e+11,NaN,NaN,1.413950e+11,1.021800e+10,1.412400e+10,2.694400e+10
6,2013-09-28,1.425900e+10,0.327,2.628700e+10,4.054600e+10,2.064100e+10,1.764000e+09,7.328600e+10,1.659700e+10,5.756000e+09,...,8.345100e+10,-4.710000e+08,1.042560e+11,1.235490e+11,NaN,NaN,1.325020e+11,-2.358600e+10,1.033500e+10,2.129100e+10
7,2012-09-29,1.074600e+10,0.095,1.838300e+10,2.912900e+10,1.869200e+10,7.910000e+08,5.765300e+10,1.545200e+10,5.359000e+09,...,5.785400e+10,4.990000e+08,1.012890e+11,1.182100e+11,NaN,NaN,1.105050e+11,-2.912900e+10,9.041000e+09,1.736700e+10
8,2011-09-24,9.815000e+09,-0.128,1.613700e+10,2.595200e+10,1.171700e+10,7.760000e+08,4.498800e+10,7.777000e+09,4.432000e+09,...,3.975600e+10,4.430000e+08,6.284100e+10,7.661500e+10,NaN,NaN,7.175500e+10,-2.595200e+10,6.543000e+09,1.333800e+10
9,2010-09-25,1.126100e+10,1.140,1.435900e+10,2.562000e+10,9.924000e+09,1.051000e+09,4.167800e+10,4.768000e+09,1.083000e+09,...,2.739200e+10,-4.600000e+07,3.716900e+10,4.779100e+10,NaN,NaN,3.975000e+10,-2.562000e+10,5.083000e+09,8.707000e+09
